In [1]:
import pandas as pd
import numpy as np
import pandas as pd 
import numpy as np
pd.options.display.max_columns = None
from matplotlib import pyplot as plt 
%matplotlib inline 
import seaborn as sns
from sklearn import preprocessing, svm 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import math
import statsmodels.formula.api as smf


C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [310]:
df = pd.read_csv('tokyo1.csv')

In [311]:
df.isna().sum()

country_name    0
Station         0
quarter_key     0
n_deals         0
n_merchants     0
nob             0
nor             0
orders          0
c_purchasers    0
c_uvv           0
Flag            0
dtype: int64

In [401]:
class locn_pritsn_report:
    def __init__(self, df):
        df['pc'] =df['c_purchasers']/df['c_uvv']
        df['pf'] =df['orders']/df['c_purchasers']
        df['log_merchants'] = np.log(df['n_merchants'])
        
        self.df_m=df[df['Flag']=='existing'].reset_index(drop=True).copy()
        self.df=df.copy()
        
        self.pc_model()
        self.pf_model()
        
    def pc_model(self):
        PC = pd.concat([self.df_m['c_purchasers'],self.df_m['c_uvv']-self.df_m['c_purchasers']],axis=1)
        formula ='PC ~ n_merchants+Station'
        pc_model = smf.glm(formula = formula, data=self.df_m, family=sm.families.Binomial())
        pc_result = pc_model.fit()
        if pc_result.pvalues['n_merchants']<=0.05:
            self.pc_merchant  = pc_result.params['n_merchants']
        else:
            self.pc_merchant=0
        self.pc_pseudoR2=1-pc_result.deviance/pc_result.null_deviance

        
    def pf_model(self):
        formula='np.log(pf) ~ log_merchants+Station'
        pf_model = smf.ols(formula,data=self.df_m)
        pf_results = pf_model.fit()
        if pf_results.pvalues['log_merchants']<=0.05:
            self.pf_merchant= pf_results.params['log_merchants']
        else:
            self.pf_merchant=0

            
    def pivoting(self):
        df=self.df.groupby(['Station', 'quarter_key'])['nob', 'c_uvv','c_purchasers','n_merchants','orders'].sum().reset_index(drop=False).copy()
        df['pf']           =  df['orders']/df['c_purchasers']
        df['pc']           =  df['c_purchasers']/df['c_uvv']
        df['orders/uvv']   =  df['orders']/df['c_uvv']
        
        def pc_pred(x):
            try:
                return(1/(1+math.exp(-(10*self.pc_merchant + math.log(x/(1-x))))))
            except:
                return 0
            
        def pf_pred(x):
            try:
                return(x['pf']*(((x['n_merchants']+10)/x['n_merchants'])**self.pf_merchant))
            except:
                return 0
        
        df['pc_10merch']   =  df['pc'].apply(lambda x: pc_pred(x) )
        df['pf_10merch']   =  df[['pf','n_merchants']].apply(pf_pred, axis=1)
        
        df[['pf','pc', 'orders/uvv', 'pc_10merch', 'pf_10merch']]=df[['pf','pc', 'orders/uvv', 'pc_10merch', 'pf_10merch']].fillna(0,axis=0)
                
        df['p_orders/uvv']=df['pc_10merch']*df['pf_10merch']
        df['pred_orders']=np.round(df['pc_10merch']*df['pf_10merch']*df['c_uvv'])
        
        #Orders Model
        df['incr_orders']=df['pred_orders']-df['orders']
        df['incr_orders']=df['incr_orders'].apply(lambda x: x if x>=0 else 0)
        df['incr_orders_rnk'] =df.groupby('quarter_key')['incr_orders'].rank(method='min', ascending=False)
        
        #weighted Model
        df['nob_rnk'] =df.groupby('quarter_key')['nob'].rank(method='min', ascending=False)
        df['uvv_rnk'] =df.groupby('quarter_key')['c_uvv'].rank(method='min', ascending=False)
        df['pc_rnk']  =df.groupby('quarter_key')['pc'].rank(method='min', ascending=False)
        df['pf_rnk']  =df.groupby('quarter_key')['pf'].rank(method='min', ascending=False)
        
        df['wtd_rnk']       =(df['nob_rnk']*0.0) + (df['uvv_rnk']*0.5) + (df['pc_rnk']*0.3) + (df['pf_rnk']*0.2)
        df['wtd_final_rnk'] =df.groupby('quarter_key')['wtd_rnk'].rank(method='min', ascending=True)
        
        
        #NOB model
        df['incr_NOB']=(df['nob']/df['orders'])*df['incr_orders']
        df['incr_NOB']=df[['incr_NOB']].fillna(0, axis=0)
        
        df['incr_nob_rnk']=df.groupby('quarter_key')['incr_NOB'].rank(method='min', ascending=False)
        tmp_df=df.sort_values('incr_NOB', ascending=False).copy()
        
        tmp_df['incr_nob_cum'] = tmp_df.groupby('quarter_key')['incr_NOB'].cumsum()
        
        tmp = df.groupby('quarter_key')['incr_NOB'].sum().reset_index()
        tmp.rename(columns={'incr_NOB':'incr_nob_sumtotal'}, inplace=True)
       
        tmp_df = pd.merge(tmp_df,tmp,how='left',on='quarter_key').copy()
        tmp_df['incnob_cum_perc'] = (tmp_df['incr_nob_cum']/tmp_df['incr_nob_sumtotal'])*100
        
        df=df.merge(tmp_df[['quarter_key','Station', 'incr_nob_cum','incr_nob_sumtotal','incnob_cum_perc']], how='left', on=['quarter_key','Station'])

        
        #Aggregation
        df=pd.pivot_table(df, columns='quarter_key', index='Station')
        
        df['orders_topzip']=pd.DataFrame((df[('incr_orders_rnk', 201900001)]<=20) | (df[('incr_orders_rnk', 201900002)]<=20) |
               (df[('incr_orders_rnk', 201900003)]<=20) | (df[('incr_orders_rnk', 201900004)]<=20))
        df['nob_topzip']=pd.DataFrame((df[('incr_orders_rnk', 201900001)]<=20) | (df[('incr_nob_rnk', 201900002)]<=20) |
               (df[('incr_nob_rnk', 201900003)]<=20) | (df[('incr_nob_rnk', 201900004)]<=20))
        df['wtd_topzip']=pd.DataFrame((df[('wtd_final_rnk', 201900001)]<=20) | (df[('wtd_final_rnk', 201900002)]<=20) |
               (df[('wtd_final_rnk', 201900003)]<=20) | (df[('wtd_final_rnk', 201900004)]<=20))

        
        df['topzip_all_approaches']=pd.DataFrame((df['orders_topzip']) &(df['nob_topzip']) & (df['wtd_topzip']))
        
        
        return df
#         tmp=df.groupby(['Station', 'quarter_key'])['nob'].rank(method='min', ascending=False).reset_index(drop=False).copy()
        
        
        
    
#         iferror(exp(LN(F361/(1-F361))+$AG$1*Regressions!$E$5)/(1+exp(LN(F361/(1-F361))+$AG$1*Regressions!$E$5)),0)


In [402]:
lc=locn_pritsn_report(df)

C:\Users\SirajuddinMohammed\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\statsmodels\base\data.py:480: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if hasattr(x, 'ix'):


In [403]:
a=(lc.pivoting())


C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in double_scalars


In [404]:
a.head()

,Station,quarter_key,nob,c_uvv,c_purchasers,n_merchants,orders,pf,pc,orders/uvv,pc_10merch,pf_10merch,p_orders/uvv,pred_orders,incr_orders,incr_orders_rnk,nob_rnk,uvv_rnk,pc_rnk,pf_rnk,wtd_rnk,wtd_final_rnk,incr_NOB,incr_nob_rnk,incr_nob_cum,incr_nob_sumtotal,incnob_cum_perc
0,Adachi Kodai,201900001,3387.41,2797.0,107.0,3,121.0,1.130841,0.038255,0.043261,0.040710,1.191824,0.048519,136.0,15.0,119.0,132.0,146.0,62.0,143.0,120.2,110.0,419.926860,125.0,399170.769339,424176.197079,94.104943
1,Adachi Kodai,201900002,2810.57,2282.0,73.0,3,85.0,1.164384,0.031989,0.037248,0.034056,1.227175,0.041793,95.0,10.0,145.0,152.0,169.0,112.0,118.0,141.7,145.0,330.655294,154.0,510861.036786,528271.383765,96.704280
2,Adachi Kodai,201900003,1214.91,1548.0,52.0,1,56.0,1.076923,0.033592,0.036176,0.035758,1.173510,0.041962,65.0,9.0,170.0,199.0,195.0,103.0,182.0,164.8,177.0,195.253393,185.0,473144.798881,482012.948124,98.160184
3,Adachi Kodai,201900004,435.46,1142.0,20.0,2,20.0,1.000000,0.017513,0.017513,0.018663,1.066284,0.019900,23.0,3.0,210.0,233.0,200.0,233.0,236.0,217.1,241.0,65.319000,230.0,445161.725786,446614.254004,99.674769
4,Aihara,201900001,0.00,0.0,0.0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,270.0,288.0,292.0,288.0,288.0,290.0,292.0,0.000000,270.0,424176.197079,424176.197079,100.000000


In [407]:
d.head()

c_purchasers                                   c_uvv            \
quarter_key       201900001 201900002 201900003 201900004 201900001 201900002   
Station                                                                         
Adachi Kodai          107.0      73.0      52.0      20.0    2797.0    2282.0   
Aihara                  0.0       0.0       0.0       0.0       0.0       0.0   
Akabane               400.0     430.0     415.0     945.0   16788.0   15119.0   
Akabanebashi          276.0     276.0     421.0     298.0    9748.0   15143.0   
Akasakamitsuke       3132.0    2143.0    1777.0    2454.0  112678.0   77326.0   

                                   incnob_cum_perc                          \
quarter_key    201900003 201900004       201900001   201900002   201900003   
Station                                                                      
Adachi Kodai      1548.0    1142.0       94.104943   96.704280   98.160184   
Aihara               0.0       0.0      100.000000  100.000000  100.000000   
Akabane          14174.0   19252.0       80.309069   80.081547   74.087565   
Akabanebashi     13841.0    9656.0       87.266877   86.135337   79.942803   
Akasakamitsuke   53411.0   50967.0       48.003459   54.807651   49.656090   

                               incr_NOB                            \
quarter_key      201900004    201900001    201900002    201900003   
Station                                                             
Adachi Kodai     99.674769   419.926860   330.655294   195.253393   
Aihara          100.000000     0.000000     0.000000     0.000000   
Akabane          44.795119  1565.751747  1972.822298  2226.489231   
Akabanebashi     85.790824   974.160374  1199.131656  1573.988608   
Akasakamitsuke   51.236382  7858.173856  7580.717730  7289.775433   

                              incr_nob_cum                                \
quarter_key       201900004      201900001      201900002      201900003   
Station                                                                    
Adachi Kodai      65.319000  399170.769339  510861.036786  473144.798881   
Aihara             0.000000  424176.197079  528271.383765  482012.948124   
Akabane         8202.033252  340651.956841  423047.896293  357111.656251   
Akabanebashi    1024.167735  370165.321514  455028.337159  385334.660934   
Akasakamitsuke  6259.785675  203619.247976  289533.135031  239348.781183   

                              incr_nob_rnk                                \
quarter_key         201900004    201900001 201900002 201900003 201900004   
Station                                                                    
Adachi Kodai    445161.725786        125.0     154.0     185.0     230.0   
Aihara          446614.254004        270.0     281.0     280.0     268.0   
Akabane         200061.385638         56.0      51.0      41.0      10.0   
Akabanebashi    383154.048949         80.0      73.0      56.0      77.0   
Akasakamitsuke  228828.983897         11.0      13.0      11.0      14.0   

               incr_nob_sumtotal                                               \
quarter_key            201900001      201900002      201900003      201900004   
Station                                                                         
Adachi Kodai       424176.197079  528271.383765  482012.948124  446614.254004   
Aihara             424176.197079  528271.383765  482012.948124  446614.254004   
Akabane            424176.197079  528271.383765  482012.948124  446614.254004   
Akabanebashi       424176.197079  528271.383765  482012.948124  446614.254004   
Akasakamitsuke     424176.197079  528271.383765  482012.948124  446614.254004   

               incr_orders                               incr_orders_rnk  \
quarter_key      201900001 201900002 201900003 201900004       201900001   
Station                                                                    
Adachi Kodai         15.00     10.00      9.00      3.00           119.0   
Aihara           

In [467]:
d.head()

c_purchasers                                   c_uvv            \
quarter_key       201900001 201900002 201900003 201900004 201900001 201900002   
Station                                                                         
Adachi Kodai          107.0      73.0      52.0      20.0    2797.0    2282.0   
Aihara                  0.0       0.0       0.0       0.0       0.0       0.0   
Akabane               400.0     430.0     415.0     945.0   16788.0   15119.0   
Akabanebashi          276.0     276.0     421.0     298.0    9748.0   15143.0   
Akasakamitsuke       3132.0    2143.0    1777.0    2454.0  112678.0   77326.0   

                                   incnob_cum_perc                          \
quarter_key    201900003 201900004       201900001   201900002   201900003   
Station                                                                      
Adachi Kodai      1548.0    1142.0       94.104943   96.704280   98.160184   
Aihara               0.0       0.0      100.000000  100.000000  100.000000   
Akabane          14174.0   19252.0       80.309069   80.081547   74.087565   
Akabanebashi     13841.0    9656.0       87.266877   86.135337   79.942803   
Akasakamitsuke   53411.0   50967.0       48.003459   54.807651   49.656090   

                               incr_NOB                            \
quarter_key      201900004    201900001    201900002    201900003   
Station                                                             
Adachi Kodai     99.674769   419.926860   330.655294   195.253393   
Aihara          100.000000     0.000000     0.000000     0.000000   
Akabane          44.795119  1565.751747  1972.822298  2226.489231   
Akabanebashi     85.790824   974.160374  1199.131656  1573.988608   
Akasakamitsuke   51.236382  7858.173856  7580.717730  7289.775433   

                              incr_nob_cum                                \
quarter_key       201900004      201900001      201900002      201900003   
Station                                                                    
Adachi Kodai      65.319000  399170.769339  510861.036786  473144.798881   
Aihara             0.000000  424176.197079  528271.383765  482012.948124   
Akabane         8202.033252  340651.956841  423047.896293  357111.656251   
Akabanebashi    1024.167735  370165.321514  455028.337159  385334.660934   
Akasakamitsuke  6259.785675  203619.247976  289533.135031  239348.781183   

                              incr_nob_rnk                                \
quarter_key         201900004    201900001 201900002 201900003 201900004   
Station                                                                    
Adachi Kodai    445161.725786        125.0     154.0     185.0     230.0   
Aihara          446614.254004        270.0     281.0     280.0     268.0   
Akabane         200061.385638         56.0      51.0      41.0      10.0   
Akabanebashi    383154.048949         80.0      73.0      56.0      77.0   
Akasakamitsuke  228828.983897         11.0      13.0      11.0      14.0   

               incr_nob_sumtotal                                               \
quarter_key            201900001      201900002      201900003      201900004   
Station                                                                         
Adachi Kodai       424176.197079  528271.383765  482012.948124  446614.254004   
Aihara             424176.197079  528271.383765  482012.948124  446614.254004   
Akabane            424176.197079  528271.383765  482012.948124  446614.254004   
Akabanebashi       424176.197079  528271.383765  482012.948124  446614.254004   
Akasakamitsuke     424176.197079  528271.383765  482012.948124  446614.254004   

               incr_orders                               incr_orders_rnk  \
quarter_key      201900001 201900002 201900003 201900004       201900001   
Station                                                                    
Adachi Kodai         15.00     10.00      9.00      3.00           119.0   
Aihara           

In [461]:
(pd.DataFrame(d.isna().sum()))

0
              quarter_key   
c_purchasers  201900001    0
              201900002    0
              201900003    0
              201900004    0
c_uvv         201900001    0
...                       ..
wtd_rnk       201900001    0
              201900002    0
              201900003    0
              201900004    0
orders_topzip              0

[101 rows x 1 columns]

In [446]:
d.col

,c_purchasers
quarter_key,201900001
Station,
Adachi Kodai,107.0
Aihara,0.0
Akabane,400.0
Akabanebashi,276.0
Akasakamitsuke,3132.0
...,...
hometown,0.0
lotus root,0.0


In [291]:
a.groupby('quarter_key')['pc'].rank(method='min', ascending=False)

0        62.0
1       112.0
2       103.0
3       233.0
4       167.0
        ...  
1168     49.0
1169     95.0
1170    224.0
1171    177.0
1172    141.0
Name: pc, Length: 1173, dtype: float64

In [253]:
(a['nob_rnk']*0.0)# + 
print(a['uvv_rnk']*0.5)# + 
print(a['pc_rnk']*0.3)# + 
print(a['pf_rnk']*0.2)

0        73.0
1        84.5
2        97.5
3       100.0
4        21.5
        ...  
1168     42.0
1169     76.5
1170     68.0
1171     68.5
1172     55.5
Name: uvv_rnk, Length: 1173, dtype: float64
0       18.6
1       33.6
2       30.9
3       69.9
4       50.1
        ... 
1168    14.7
1169    28.5
1170    67.2
1171    53.1
1172    42.3
Name: pc_rnk, Length: 1173, dtype: float64
0       28.6
1       23.6
2       36.4
3       47.2
4       26.4
        ... 
1168    41.6
1169    33.0
1170     7.2
1171     9.4
1172    18.6
Name: pf_rnk, Length: 1173, dtype: float64


In [235]:
a.head()

,Station,quarter_key,nob,c_uvv,c_purchasers,n_merchants,orders,pf,pc,orders/uvv,pc_10merch,pf_10merch,p_orders/uvv,pred_orders,incr_orders
0,Adachi Kodai,201900001,3387.41,2797.0,107.0,3,121.0,1.130841,0.038255,0.043261,0.040710,1.191824,0.048519,135.706696,15.0
1,Adachi Kodai,201900002,2810.57,2282.0,73.0,3,85.0,1.164384,0.031989,0.037248,0.034056,1.227175,0.041793,95.371013,10.0
2,Adachi Kodai,201900003,1214.91,1548.0,52.0,1,56.0,1.076923,0.033592,0.036176,0.035758,1.173510,0.041962,64.957679,9.0
3,Adachi Kodai,201900004,435.46,1142.0,20.0,2,20.0,1.000000,0.017513,0.017513,0.018663,1.066284,0.019900,22.725296,3.0
4,Akabane,201900001,17074.15,16788.0,400.0,10,458.0,1.145000,0.023827,0.027281,0.025380,1.173784,0.029790,500.117166,42.0


In [292]:
df.Flag.unique()

array(['existing', 'only Multi-zip', 'no-deal'], dtype=object)

In [125]:
df[df['c_uvv']==0]

,country_name,Station,quarter_key,n_deals,n_merchants,nob,nor,orders,c_purchasers,c_uvv,Flag,pc,pf,log_merchants
125,JP,Eifukucho,201900004,1,1,0.0,0.0,0.0,0.0,0.0,only Multi-zip,NaN,NaN,0.0
142,JP,Fuchuhonmachi,201900001,1,1,0.0,0.0,0.0,0.0,0.0,only Multi-zip,NaN,NaN,0.0
143,JP,Fuchuhonmachi,201900002,1,1,0.0,0.0,0.0,0.0,0.0,only Multi-zip,NaN,NaN,0.0
144,JP,Fuchuhonmachi,201900003,1,1,0.0,0.0,0.0,0.0,0.0,only Multi-zip,NaN,NaN,0.0
145,JP,Fuchuhonmachi,201900004,1,1,0.0,0.0,0.0,0.0,0.0,only Multi-zip,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,JP,lotus root,201900004,0,0,0.0,0.0,0.0,0.0,0.0,no-deal,NaN,NaN,-inf
1860,JP,lotus root,201900002,0,0,0.0,0.0,0.0,0.0,0.0,no-deal,NaN,NaN,-inf
1861,JP,lotus root,201900001,0,0,0.0,0.0,0.0,0.0,0.0,no-deal,NaN,NaN,-inf
1862,JP,lotus root,201900003,0,0,0.0,0.0,0.0,0.0,0.0,no-deal,NaN,NaN,-inf


In [124]:
try:
    print(1)
except:
    print('cool')

1
